In [168]:
#coding:utf-8
import pandas as pd
import numpy as np
import time
import datetime
import gc
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

In [169]:
path = '/Users/apple/Desktop/data/basicclick_data/'
data = pd.read_csv(path+"rate_final_data_12-01-10-41.csv")

In [170]:
data.shape

(10184, 74)

In [171]:
len(data.columns)

74

In [172]:
train = pd.read_csv(path+"train_day7.csv")
train = train[['instance_id']]
train

,instance_id
0,7.608450e+18
1,4.572110e+18
2,8.091750e+18
3,7.251110e+18
4,5.545850e+18
...,...
8142,5.568580e+18
8143,2.780010e+18
8144,2.230740e+18
8145,2.144820e+18


In [173]:
# 合并时指定左右键，确保两边的键相同
train = pd.merge(train, data, how='left', left_on='instance_id', right_on='instance_id')


In [174]:
train = train.drop('day_y', axis=1)
train = train.drop('is_trade_y', axis=1)
train = train.rename(columns={'day_x': 'day'})
train = train.rename(columns={'is_trade_x': 'is_trade'})

In [176]:
train.columns

Index(['instance_id', 'item_id', 'item_brand_id', 'item_city_id',
       'item_price_level', 'item_sales_level', 'item_collected_level',
       'item_pv_level', 'user_id', 'user_gender_id', 'user_age_level',
       'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'shop_id',
       'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level',
       'shop_score_service', 'shop_score_delivery', 'shop_score_description',
       'is_trade', 'time', 'day', 'hour', 'minute', 'maphour', 'mapmin',
       'item_category_0', 'item_category_1', 'item_category_2',
       'item_maphour_cnt_ratio', 'user_hour_cnt_ratio',
       'user_maphour_cnt_ratio', 'same_time_expo_cnt', 'user_mean_hour',
       'item_mean_hour', 'brand_mean_hour', 'shop_mean_hour', 'user_mean_age',
       'item_mean_age', 'brand_mean_age', 'shop_mean_age', 'user_active_hour',
       'user_day_active_city', 'item_day_active_user', 'shop_day_active_user',
       'bra

In [177]:
def feat_select(train, test):
  
    features = train.drop(['is_trade', 'time', 'day','context_timestamp','user_id','context_id','instance_id','item_id','shop_id','item_brand_id'], axis=1).columns.tolist() #,'item_property_list','predict_category_property'

    target = ['is_trade']

    return features, target

In [178]:
def lgbCV(train, test, All):
    features, target = feat_select(train, test)

    lgb_clf = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=48, max_depth=-1, learning_rate=0.05)
    # lgb_clf = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=48, max_depth=-1, learning_rate=0.05, n_estimators=2000,
    #                            max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,
    #                            min_child_weight=5, min_child_samples=10, subsample=1, subsample_freq=1,
    #                            colsample_bytree=1, reg_alpha=3, reg_lambda=5, seed=1000, nthread=-1, silent=True)

    lgb_model = lgb_clf.fit(train[features], train[target], eval_set=[(test[features], test[target])])
    best_iter = lgb_model.best_iteration_ 
    
    # 特征重要性
    lgb_predictors = [i for i in train[features].columns]
    lgb_feat_imp = pd.Series(lgb_model.feature_importances_, lgb_predictors).sort_values(ascending=False)
    lgb_feat_imp.to_csv('lgb_feat_imp_day7.csv')
    
    # 训练模型
    lgb_clf.fit(train[features], train[target])
    test['lgb_predict'] = lgb_clf.predict_proba(test[features])[:, 1]
    lgb_loss = log_loss(test[target], test['lgb_predict'])  
    lgb_auc = roc_auc_score(test[target], test['lgb_predict'])
    print('Training loss: %.6f, Training AUC: %.6f' % (lgb_loss, lgb_auc))
    
    return best_iter,lgb_loss

In [179]:
test = pd.read_csv(path+"test_day7.csv")
test = test[['instance_id']]
test = pd.merge(test, data, how='left', on='instance_id')

In [180]:

test = test.drop('day_y', axis=1)
test = test.drop('is_trade_y', axis=1)
test = test.rename(columns={'day_x': 'day'})
test = test.rename(columns={'is_trade_x': 'is_trade'})

In [181]:
test.columns

Index(['instance_id', 'item_id', 'item_brand_id', 'item_city_id',
       'item_price_level', 'item_sales_level', 'item_collected_level',
       'item_pv_level', 'user_id', 'user_gender_id', 'user_age_level',
       'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'shop_id',
       'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level',
       'shop_score_service', 'shop_score_delivery', 'shop_score_description',
       'is_trade', 'time', 'day', 'hour', 'minute', 'maphour', 'mapmin',
       'item_category_0', 'item_category_1', 'item_category_2',
       'item_maphour_cnt_ratio', 'user_hour_cnt_ratio',
       'user_maphour_cnt_ratio', 'same_time_expo_cnt', 'user_mean_hour',
       'item_mean_hour', 'brand_mean_hour', 'shop_mean_hour', 'user_mean_age',
       'item_mean_age', 'brand_mean_age', 'shop_mean_age', 'user_active_hour',
       'user_day_active_city', 'item_day_active_user', 'shop_day_active_user',
       'bra

In [182]:
All = True
best_iter, lgb_loss = lgbCV(train, test, All)

/Users/apple/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/apple/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/Users/apple/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 366, number of negative: 7781
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003156 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3185
[LightGBM] [Info] Number of data points in the train set: 8147, number of used features: 56
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.044925 -> initscore=-3.056807
[LightGBM] [Info] Start training from score -3.056807


/Users/apple/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/apple/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 366, number of negative: 7781
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003134 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3185
[LightGBM] [Info] Number of data points in the train set: 8147, number of used features: 56
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.044925 -> initscore=-3.056807
[LightGBM] [Info] Start training from score -3.056807
Training loss: 0.181858, Training AUC: 0.579639


In [183]:
best_iter

0

In [184]:
lgb_loss

0.18185776702447454